In [29]:
import pandas as pd
import numpy as np
from scipy import stats
from math import nan
from sqlalchemy import create_engine
import pyodbc
import os
import re

In [2]:
os.getcwd()

'C:\\Users\\truong le'

In [3]:
os.chdir('D:\school\làm việc\Yes4All')

In [42]:
raw = pd.read_excel('data_for_forecast.xlsx')
raw.head()

,week,sku,quantity,daily_qnt,inv_amz,inv_y4a,qnt_oos
0,9,8ACX,17.0,2.1408,74.0,618.0,17.0
1,10,8ACX,34.0,2.1408,62.5,574.5,34.0
2,11,8ACX,31.0,2.1408,42.0,520.0,31.0
3,12,8ACX,33.0,2.1408,22.0,499.5,33.0
4,13,8ACX,55.0,2.1408,16.5,446.0,55.0


In [43]:
info = pd.read_excel('Basic_Product_Info_Report-2021-09-09.xlsx')
info.head()

,sku,Title,Company,Life_cycle,Agent_vendor
0,01JV,Supply WH - Brown Box - Dumbbell Neoprene - 10...,Y4A,WH Supply - Temp,Jason Agent
1,02MN,Weight Plate 1in - 10 lbs - 4 pcs,Y4A,Normal,Jason Agent
2,03NF,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal,Roger Agent
3,07AG,Agility Hex Ring - 12 Rings,Y4A,Normal,Jason Agent
4,07V0,Wood Bag - Black / Grey,Y4A,Obsoleted,Roger Agent


In [44]:

full_data = pd.merge(raw, info[['sku','Title','Company','Life_cycle']], on = 'sku', how = 'left')
full_data.head()

,week,sku,quantity,daily_qnt,inv_amz,inv_y4a,qnt_oos,Title,Company,Life_cycle
0,9,8ACX,17.0,2.1408,74.0,618.0,17.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
1,10,8ACX,34.0,2.1408,62.5,574.5,34.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
2,11,8ACX,31.0,2.1408,42.0,520.0,31.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
3,12,8ACX,33.0,2.1408,22.0,499.5,33.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
4,13,8ACX,55.0,2.1408,16.5,446.0,55.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal


In [45]:
full_data[full_data['Life_cycle'].isnull()]

,week,sku,quantity,daily_qnt,inv_amz,inv_y4a,qnt_oos,Title,Company,Life_cycle


In [46]:
#filter out mask and HMD
df = full_data[(~full_data['Title'].str.contains('Mask')) & (~full_data['Company'].str.contains('HMD'))]
df

,week,sku,quantity,daily_qnt,inv_amz,inv_y4a,qnt_oos,Title,Company,Life_cycle
0,9,8ACX,17.0,2.1408,74.0,618.0,17.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
1,10,8ACX,34.0,2.1408,62.5,574.5,34.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
2,11,8ACX,31.0,2.1408,42.0,520.0,31.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
3,12,8ACX,33.0,2.1408,22.0,499.5,33.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
4,13,8ACX,55.0,2.1408,16.5,446.0,55.0,Kids Puzzle - 9 colour - 36 tiles,Y4A,Withdrawal
...,...,...,...,...,...,...,...,...,...,...
25078,31,CTZ7,NaN,32.3333,NaN,206.0,0.0,Suspension Straps - Yellow,Y4A,Withdrawal
25079,32,CTZ7,NaN,32.3333,NaN,206.0,0.0,Suspension Straps - Yellow,Y4A,Withdrawal
25080,33,CTZ7,NaN,32.3333,NaN,206.0,0.0,Suspension Straps - Yellow,Y4A,Withdrawal
25081,34,CTZ7,NaN,32.3333,NaN,206.0,0.0,Suspension Straps - Yellow,Y4A,Withdrawal


In [47]:
#filter life cycle
lc = ['Turnkey','Normal','1st year Growth', 'Introduction']
df = df[df['Life_cycle'].isin(lc)]
df

,week,sku,quantity,daily_qnt,inv_amz,inv_y4a,qnt_oos,Title,Company,Life_cycle
27,9,KQR2,9.0,2.3764,0.0,751.0,9.0000,Macebell 10lb - New,Y4A,Normal
28,10,KQR2,34.0,2.3764,0.0,731.0,34.0000,Macebell 10lb - New,Y4A,Normal
29,11,KQR2,39.0,2.3764,0.0,789.5,39.0000,Macebell 10lb - New,Y4A,Normal
30,12,KQR2,51.0,2.3764,31.0,655.0,51.0000,Macebell 10lb - New,Y4A,Normal
31,13,KQR2,38.0,2.3764,76.0,646.0,38.0000,Macebell 10lb - New,Y4A,Normal
...,...,...,...,...,...,...,...,...,...,...
24835,31,G2M8,NaN,1.1377,NaN,3.0,7.9639,Slam Ball - Triangle - Black - 10lbs,Y4A,Normal
24836,32,G2M8,NaN,1.1377,NaN,3.0,7.9639,Slam Ball - Triangle - Black - 10lbs,Y4A,Normal
24837,33,G2M8,NaN,1.1377,NaN,NaN,7.9639,Slam Ball - Triangle - Black - 10lbs,Y4A,Normal
24838,34,G2M8,NaN,1.1377,NaN,NaN,7.9639,Slam Ball - Triangle - Black - 10lbs,Y4A,Normal


In [48]:
df = pd.pivot_table(df,values='qnt_oos', index = ['week'], columns = ['sku'])

In [49]:
df = pd.DataFrame(df.to_records())
df.head(10)

,week,9,02MN,03NF,07AG,09SM,0JJV,1ARL,1J6H,1NHS,...,ZR6J,ZRLX,ZS2H,ZSM8,ZSQM,ZT2X,ZVTJ,ZXSV,ZYLU,ZZCE
0,9,156.0,0.000,2.0,8.0,27.0,18.0,15.0,40.0,5.0000,...,12.0043,7.0,130.0,13.0,7.0,6.0,37.0,143.2032,16.0000,7.014
1,10,235.0,0.000,4.0,33.0,91.0,60.0,34.0,132.0,10.0000,...,12.0043,18.0,439.0,22.0,7.0,5.0,82.0,143.2032,32.0000,7.014
2,11,67.0,7.000,6.0,32.0,56.0,197.0,47.0,135.0,23.3849,...,12.0043,27.0,354.0,29.0,7.0,11.0,95.0,143.2032,42.0000,7.014
3,12,309.0,8.000,13.0,40.0,60.0,177.0,50.0,136.0,5.0000,...,12.0043,14.0,233.0,26.0,7.0,12.0,98.0,143.2032,42.0000,7.014
4,13,299.0,8.512,15.0,32.0,33.0,55.0,29.0,109.0,23.3849,...,12.0043,14.0,95.0,16.0,7.0,4.0,105.0,143.2032,30.0000,7.014
5,14,194.0,8.512,27.0,33.0,16.0,1.0,27.0,127.0,23.3849,...,12.0043,14.0,39.0,12.0,7.0,9.0,59.0,143.2032,19.0000,7.014
6,15,135.0,8.512,32.0,38.0,10.0,186.0,30.0,101.0,23.3849,...,12.0043,14.0,11.0,6.0,7.0,21.0,74.0,143.2032,1.0000,7.014
7,16,154.0,8.512,19.0,35.0,22.0,168.0,36.0,117.0,23.3849,...,12.0043,14.0,0.0,6.0,7.0,13.0,54.0,70.0000,1.0000,7.014
8,17,114.0,8.512,24.0,41.0,28.0,84.0,37.0,65.0,23.3849,...,12.0043,14.0,1.0,8.0,7.0,7.0,100.0,30.0000,17.0261,7.014
9,18,146.0,8.512,40.0,42.0,46.0,234.0,22.0,62.0,23.3849,...,12.0043,14.0,2.0,7.0,7.0,5.0,106.0,54.0000,17.0261,7.014


In [50]:
def find_new_x (x,j,col):
    z = 3
    done = False
    mean = df[col].mean()
    std = df[col].std()
    while done == False:
        x = mean + z*std
        df[col][j] = x
        if df[col].std() == 0:
            z = z - 0.25
        else:
            z_new = (x - df[col].mean())/df[col].std()
            if z_new >= 3:
                z = z - 0.25
            else:
                done = True
    return(x)

In [51]:
def find_new_x_n (x,j,col):
    z = -3
    done = False
    mean = df[col].mean()
    std = df[col].std()
    while done == False:
        x = mean + z*std
        df[col][j] = x
        if df[col].std() == 0:
            z = z + 0.25
        else:
            z_new = (x - df[col].mean())/df[col].std()
            if z_new <= -3:
                z = z + 0.25
            else:
                done = True
    return(x)

In [52]:
def cleancol(col):
    restart = True
    while restart:
        restart = False
        if (df[col].std() == 0) or (df.duplicated([col]).sum() > 0.9*len(df)):
            df[col] = df[col]
        else:
            for i in range(0,len(df)):
                if ((df.iloc[i,df.columns.get_loc(col)]- df[col].mean())/df[col].std()) > 3:
                    df.iloc[i,df.columns.get_loc(col)] = find_new_x(df[col][i],i,col)
                    restart = True
                    break
                elif ((df.iloc[i,df.columns.get_loc(col)]- df[col].mean())/df[col].std()) < -3:
                    df.iloc[i,df.columns.get_loc(col)] = find_new_x_n(df[col][i],i,col)
                    restart = True
                    break
                else:
                    df.iloc[i,df.columns.get_loc(col)] = df.iloc[i,df.columns.get_loc(col)]
    return df[col]

In [53]:
for q in df.columns[1:]:
    print(q)
    cleancol(q)

9
02MN


<ipython-input-50-b64ff87a7221>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][j] = x


03NF
07AG
09SM
0JJV
1ARL
1J6H
1NHS
1XTU
1YHQ
2KLC
36DB
37XZ
3KHB
3MX1
3VSD
3W51
3ZMT
45MH
4M82
4YHG
4Z9X
51F7
5CH9
5NRP
5SU6
60DA
63ML
64TL
673J
6B9Q
6CLE
6KWK
6LWD
7BW4
7CPK
7KMQ
7MZ2
7RBK
7XU4
8BX6
8F0P
8GHV
8K3B
8NFL
8YCR
9KSP
9PRY
9RNM
9RV3
9SWE
9TZN
9UAZ
9VRG
A0QD
A9CZ
A9Q1
AAAR
AAAS
AAAT
AAAU
AAAV
AC1G
AEA6
AJSG
AKHG
AKNK
AMNS
ANCT
ANCU
APMB
AQ16
AS5W
AVMH
AWNY
AXW9
AY14
AYPK
AZ6B
B0AL
B0RJ
B1BQ
B42T
B62R
B6D1
BAAH
BAAI
BAAW
BAAX
BACD
BCSS
BDA1
BDM8
BK27
BKVQ
BL0J
BLC7
BME4
BOE5
BP5K
BW0X
BW7V
BXGM
BXJ4
BZ4G
C1KK
C2K6
C444
C5TJ
C756
C954
CAAH
CAAI
CAWR
CAY1
CCZN
CFH8
CFL8
CGAS
CKT4
CLHZ
CLUK
CM5X
CMJ4
CMX1
CNMF
CREG
CX7L
CXCM
CXKW
CXMR
CZB5
D1IB
D1MY
D25M
D297
D2CL
D377
D37R
D5XI
D68N
D6RW
D7SL
D8FC
D8UJ
D9YB
DA0G
DABM
DAT7
DB6F
DFPW
DJ3U
DKR2
DL2Z
DMHY
DNC8
DOHL
DSAX
DU7Y
DWJE
DWP2
DWZ2
DXD3
DY4P
DYPM
DYR0
DZIK
E0KQ
E0QT
E3SK
E3VK
E470
E5UA
E833
EBGL
EDYU
EGXQ
EH7A
EJ7F
EK0M
EK64
EM1K
ENT7
ENYH
ETHR
EVPB
EVZU
EWUN
EZR0
F410
F773
FAZ7
FDXV
FEQB
FF42
FFYE
FLCD
FLE2
FLKJ
FMUV
FNUJ


In [54]:
df

,week,9,02MN,03NF,07AG,09SM,0JJV,1ARL,1J6H,1NHS,...,ZR6J,ZRLX,ZS2H,ZSM8,ZSQM,ZT2X,ZVTJ,ZXSV,ZYLU,ZZCE
0,9,156.0,0.000000,2.0,8.0000,27.0,18.0,15.0000,40.000000,5.0000,...,12.0043,7.0,130.000000,13.0000,7.0,6.000000,37.0,143.2032,16.0000,7.014
1,10,235.0,0.000000,4.0,33.0000,91.0,60.0,34.0000,132.000000,10.0000,...,12.0043,18.0,401.165228,22.0000,7.0,5.000000,82.0,143.2032,32.0000,7.014
2,11,67.0,7.000000,6.0,32.0000,56.0,197.0,47.0000,135.000000,23.3849,...,12.0043,27.0,354.000000,29.0000,7.0,11.000000,95.0,143.2032,42.0000,7.014
3,12,309.0,8.000000,13.0,40.0000,60.0,177.0,50.0000,136.000000,5.0000,...,12.0043,14.0,233.000000,26.0000,7.0,12.000000,98.0,143.2032,42.0000,7.014
4,13,299.0,8.512000,15.0,32.0000,33.0,55.0,29.0000,109.000000,23.3849,...,12.0043,14.0,95.000000,16.0000,7.0,4.000000,105.0,143.2032,30.0000,7.014
5,14,194.0,8.512000,27.0,33.0000,16.0,1.0,27.0000,127.000000,23.3849,...,12.0043,14.0,39.000000,12.0000,7.0,9.000000,59.0,143.2032,19.0000,7.014
6,15,135.0,8.512000,32.0,38.0000,10.0,186.0,30.0000,101.000000,23.3849,...,12.0043,14.0,11.000000,6.0000,7.0,21.000000,74.0,143.2032,1.0000,7.014
7,16,154.0,8.512000,19.0,35.0000,22.0,168.0,36.0000,117.000000,23.3849,...,12.0043,14.0,0.000000,6.0000,7.0,13.000000,54.0,70.0000,1.0000,7.014
8,17,114.0,8.512000,24.0,41.0000,28.0,84.0,37.0000,65.000000,23.3849,...,12.0043,14.0,1.000000,8.0000,7.0,7.000000,100.0,30.0000,17.0261,7.014
9,18,146.0,8.512000,40.0,42.0000,46.0,234.0,22.0000,62.000000,23.3849,...,12.0043,14.0,2.000000,7.0000,7.0,5.000000,106.0,54.0000,17.0261,7.014


In [55]:
df.to_csv('cleaned_data_C.csv',index = False)